Swallow-MS-7b-v0.1による系列ラベリング

In [ ]:
!pip install transformers accelerate deplacy
from transformers import AutoTokenizer,AutoModelForTokenClassification,TokenClassificationPipeline
tkz=AutoTokenizer.from_pretrained("tokyotech-llm/Swallow-MS-7b-v0.1")
mdl=AutoModelForTokenClassification.from_pretrained("KoichiYasuoka/Swallow-MS-7b-char-upos",trust_remote_code=True,device_map="auto")
class TCP(TokenClassificationPipeline):
  def check_model_type(self,supported_models):
    pass
nlp=TCP(model=mdl,tokenizer=tkz)
txt="どこの村でも当然だった。"
doc="\n".join("\t".join([str(i+1),txt[t["start"]:t["end"]],"_",t["entity"].split("|")[0]]+["_"]*5+["SpaceAfter=No"]) for i,t in enumerate(nlp(txt)))+"\n\n"
import deplacy
deplacy.serve(doc,port=None)

トークナイザを単文字化

In [ ]:
tkz=AutoTokenizer.from_pretrained("KoichiYasuoka/Swallow-MS-7b-char-upos")
nlp=TCP(model=mdl,tokenizer=tkz)
txt="どこの村でも当然だった。"
doc="\n".join("\t".join([str(i+1),txt[t["start"]:t["end"]],"_",t["entity"].split("|")[0]]+["_"]*5+["SpaceAfter=No"]) for i,t in enumerate(nlp(txt)))+"\n\n"
deplacy.serve(doc,port=None)

出力部に逆方向のBellman-Fordを追加

In [ ]:
class BFP(TokenClassificationPipeline):
  def __init__(self,**kwargs):
    import numpy
    super().__init__(**kwargs)
    x=self.model.config.label2id
    y=[k for k in x if not k.startswith("I-")]
    self.transition=numpy.full((len(x),len(x)),numpy.nan)
    for k,v in x.items():
      for j in ["I-"+k[2:]] if k.startswith("B-") else [k]+y if k.startswith("I-") else y:
        self.transition[v,x[j]]=0
  def check_model_type(self,supported_models):
    pass
  def postprocess(self,model_outputs,**kwargs):
    import numpy
    if "logits" not in model_outputs:
      return self.postprocess(model_outputs[0],**kwargs)
    m=model_outputs["logits"][0].numpy()
    e=numpy.exp(m-numpy.max(m,axis=-1,keepdims=True))
    z=e/e.sum(axis=-1,keepdims=True)
    for i in range(m.shape[0]-1,0,-1):
      m[i-1]+=numpy.nanmax(m[i]+self.transition,axis=1)
    k=[numpy.nanargmax(m[0])]
    for i in range(1,m.shape[0]):
      k.append(numpy.nanargmax(m[i]+self.transition[k[-1]]))
    w=[{"entity":self.model.config.id2label[j],"start":s,"end":e,"score":z[i,j]} for i,((s,e),j) in enumerate(zip(model_outputs["offset_mapping"][0].tolist(),k)) if s<e]
    if "aggregation_strategy" in kwargs and kwargs["aggregation_strategy"]!="none":
      for i,t in reversed(list(enumerate(w))):
        p=t.pop("entity")
        if p.startswith("I-"):
          w[i-1]["score"]=min(w[i-1]["score"],t["score"])
          w[i-1]["end"]=w.pop(i)["end"]
        elif p.startswith("B-"):
          t["entity_group"]=p[2:]
        else:
          t["entity_group"]=p
    for t in w:
      t["text"]=model_outputs["sentence"][t["start"]:t["end"]]
    return w
nlp=BFP(model=mdl,tokenizer=tkz)
txt="どこの村でも当然だった。"
doc="\n".join("\t".join([str(i+1),txt[t["start"]:t["end"]],"_",t["entity"].split("|")[0]]+["_"]*5+["SpaceAfter=No"]) for i,t in enumerate(nlp(txt)))+"\n\n"
deplacy.serve(doc,port=None)